**Converteert de TXT files naar een excel waar de pdf link te zien is en metadata. .Gemaakt naar de excel die we doorgestuurd hebben gekregen van Statistiek Vlaanderen**

In [5]:
import os
import pandas as pd
import re

# Define the folder containing the TXT files
folder_path = "ScrapeddocumentsCorneel1202"
output_excel = "parliamentary_questions.xlsx"

# List to store extracted data
data_list = []

# Regular expressions to match different fields
date_pattern = re.compile(r"datum:\s*(.+)")
uppercase_name_pattern = re.compile(r"^(van|aan)\s+([A-Z\s]+)$")  # Matches "Van NAME" or "Aan NAME"

# Loop through all TXT files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(folder_path, filename)

        # Initialize placeholders for extracted data
        title, pdf_link, thema_link, date, asker, answerer = "", "", "", "", "", ""
        themas = []

        # Read the file and extract data
        with open(file_path, "r", encoding="utf-8") as file:
            lines = file.readlines()

            for i, line in enumerate(lines):
                line = line.strip()

                # Extract title
                if line.startswith("title:"):
                    title = line.replace("title:", "").strip()

                # Extract thema (multiple lines possible)
                elif line.startswith("thema:"):
                    themas = []
                    for j in range(i, len(lines)):
                        thema_line = lines[j].strip()
                        if thema_line and not thema_line.startswith(("pdf link:", "thema link:")):
                            themas.append(thema_line.replace("thema:", "").strip())
                        else:
                            break  # Stop if we reach another section

                # Extract PDF link
                elif line.startswith("pdf link:"):
                    pdf_link = line.replace("pdf link:", "").strip()

                # Extract Thema link
                elif line.startswith("thema link:"):
                    thema_link = line.replace("thema link:", "").strip()

                # Extract date
                elif "datum:" in line:
                    match = date_pattern.search(line)
                    if match:
                        date = match.group(1).strip()

                # Extract Question Asker (Vraagsteller) - must be uppercase after "Van"
                elif line.startswith("van "):
                    match = uppercase_name_pattern.match(line)
                    if match:
                        asker = match.group(2).strip()

                # Extract Question Answerer (Ondervraagde Minister) - must be uppercase after "Aan"
                elif line.startswith("aan "):
                    match = uppercase_name_pattern.match(line)
                    if match:
                        answerer = match.group(2).strip()

        # Store the extracted data in a list
        data_list.append({
            "Title": title,
            "Thema": ", ".join(thematic for thematic in themas),  # Store multiple themas in one block
            "PDF Link": pdf_link,
            "Thema Link": thema_link,
            "Date": date,
            "Question Asker": asker,
            "Question Answerer": answerer
        })

# Convert to DataFrame
df = pd.DataFrame(data_list)

# Save DataFrame to Excel
df.to_excel(output_excel, index=False)

print(f"✅ Extraction complete! Data saved to {output_excel}")


✅ Extraction complete! Data saved to parliamentary_questions.xlsx
